<a href="https://colab.research.google.com/github/AbdalazizOth/Tradtional_dict/blob/main/Taj_Arus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[TajArus_dict](https://github.com/OpenITI/1225AH/tree/master/data/1205MurtadaZabidi/1205MurtadaZabidi.TajCarus)**

In [ ]:
import re                               # RegEx module for regular expressions
import pandas as pd                     # Python library for data manipulation
from openpyxl import load_workbook      # Python library to read/write Excel files
from openpyxl.styles import Alignment

input_file = "/content/1205MurtadaZabidi.TajCarus.Shamela0007030-ara1"
output_file_txt = "TajArus.txt"
output_file_xlsx = "TajArus.xlsx"

# Page number pattern like PageV01P660
page_pattern = re.compile(r"PageV\d{2}P\d+")

# Root pattern (starting with ### | and not starting with parentheses)
root_pattern = re.compile(r"^### \|\s*(?!\s*\().+")

page = None             # Current active page
next_page = None        # Temporarily stores page found inside text until next root
root = None
text_lines = []
data = []
root_started = False    # Tracks if we are inside a root

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        line = line.rstrip()

        # Detect page marker
        if match := page_pattern.search(line):
            if root_started == False:

                page = match.group(0)          # Normal case: page appears before root
            else:

                next_page = match.group(0)     # New page appears inside text → belongs to next root
            continue


        # Detect new root
        if match := root_pattern.match(line):

            if root is not None:                       # Save previous root if exists
                text = " ".join(text_lines).strip()
                data.append((page, root, text))


            # Use any saved "next_page" for this new root
            if next_page is not None:
                page = next_page
                next_page = None

            # Extract new root
            root = match.group(0).replace("### |", "").strip()
            text_lines = []
            root_started = True  # now inside a new root
            continue

        # Collect text lines under current root
        if root and line.strip():
            line=re.sub(r"\bms\d{5}\b|«\d+»|\#|\~\~|\(|\)|\{|\}|\|\||\.\.\.|\s+\:|\)\:", "",line)
            text_lines.append(line.strip())

# Save last root
if root and text_lines:
    text = " ".join(text_lines).strip()
    data.append((page, root, text))
    root_started = False



# 1️⃣ Save text file
with open(output_file_txt, "w", encoding="utf-8") as out:
    for page, root, text in data:
        out.write(f"Page: {page}\nRoot: {root}\nText: {text}\n\n")

# 2️⃣ Save Excel file
df = pd.DataFrame(data, columns=["Page", "Root", "Text"])
df.to_excel(output_file_xlsx, index=False, engine="openpyxl")

# 3️⃣ Format Excel (RTL + wrap text)
wb = load_workbook(output_file_xlsx)
ws = wb.active

# Make sheet right-to-left
ws.sheet_view.rightToLeft = True

# Apply text alignment and wrapping for all cells
align = Alignment(wrap_text=True, horizontal="right", vertical="top")
for row in ws.iter_rows():
    for cell in row:
        cell.alignment = align

# Adjust column width (fixed reasonable size)
ws.column_dimensions["A"].width = 15   # Page
ws.column_dimensions["B"].width = 20   # Root
ws.column_dimensions["C"].width = 80   # Text


wb.save(output_file_xlsx)

print(f"✅ Done! {len(data)} roots extracted.")
print(f"📄 Text file: {output_file_txt}")
print(f"📊 Excel file: {output_file_xlsx}")


✅ Done! 12447 roots extracted.
📄 Text file: TajArus.txt
📊 Excel file: TajArus.xlsx
